<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D1_3_callbacks.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Callbacks

En este notebook se muestran ejemplos de formas de trabajar más eficientemente con TensorFlow y Keras.

Antes que nada, importamos las bibliotecas necesarias.

In [ ]:
# Se incluyen las bibliotecas necesarias
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use('default')

# Controlar cunado se termina el entrenamiento

Normalmente, determinar parámetros como la cantidad de *epochs* es muy inconveniente, sobre todo cuando entrenamos con un número muy bajo, pues hay que volver a entrenar el modelo y evaluar su comportamiento en las funciones de costo y métricas de calidad. Una estrategia válida (pero costosa) es entrenar el modelo por **muchisimos** *epochs*, intentando que el modelo llegue a hacer *overfitting*, y simplemente extraer lo aprendido por el modelo **antes** de que el *overfitting* sea evidente. Si bien es una estrategia efectiva, es tan costosa que se vuelve rápidamente inviable. 

Keras nos da una alternativa. Podemos usar una función para determinar si el resultado del aprendizaje nos es satisfactorio, y terminar el entrenamiento de forma temprana. Esto es un *callback*, es decir, una función que podemos definir, y que TensorFlow llama por nosotros.

El código de la siguiente celda muestra un simple callback para terminar la ejecución del programa una vez que la precisión sea mayor a 70%

In [ ]:
# heredamos de la clase `Callback` de keras
class CallbackPenguin(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.7:
            print('\nEl modelo ha llegado a 70% de precisión, terminando entrenamiento')
            self.model.stop_training = True

Vamos línea por línea.

Primero que nada, creamos una clase que hereda de `tf.keras.callbacks.Callback`. 

De momento, solo definimos la llamada `on_epoch_end`, que se llama al finalizar cada uno de los *epochs* que configuramos en nuestro modelo. Los parámetros:
* `self`: es un puntero a nuestro objeto de tipo `CallbackPenguin`, que nos permite acceder al modelo que vamos a usar más adelante.
* `epoch`: es el número de *epoch* actual.
* `logs`: es un diccionario que podemos revisar, que contiene información como la precisión, el valor de la función de costo y la precición del modelo en el *epoch* actual.

Dentro de la función, simplemente preguntamos si la precisión (accuracy, o `acc`) es mayor a 0.7 (70%), y en caso de que lo sea, imprimimos un mensaje y indicamos que deseamos terminar el entrenamiendo de forma temprana, asignando `True` a `self.model.stop_training` (variable para parar el entrenamiento).

Vea otras posibles opciones de *callbacks* en la [documentación](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback).

# Callbacks en la práctica

Veamos como usar nuestro nuevo *callback* con el modelo del ejemplo anterior.

Primero, cargamos y normalizamos el *dataset*

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images/255.0
test_images = test_images/255.0

Luego, creamos nuestro modelo

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                             tf.keras.layers.Dense(100, activation='relu'),
                             tf.keras.layers.Dense(10, activation='softmax')])

compilamos el modelo

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

Al momento de entrenar es donde necesitamos crear un objeto para nuestro *callback*, y indicamos que queremos usar el callback en el método `fit` de nuestro modelo.

In [ ]:
callback_penguin = CallbackPenguin()

model.fit(train_images, train_labels, epochs=50, callbacks=[callback_penguin])

Como puede verse al ejecutar la celda de arriba, el modelo llega a más de 70% en el primer *epoch*, y luego el entrenamiento para gracias a nuestro *callback*.

# Créditos

Este notebook utiliza y modifica recursos del [tutorial de TensorFlow](https://www.tensorflow.org/tutorials/keras/classification) y está inspirado en contenido del curso online [TensorFlow in Practice](https://www.deeplearning.ai/tensorflow-in-practice/).